In [1]:
from kafka import KafkaConsumer
import sys
import json
import time

In [19]:
# Setting up the Python consumer
bootstrap_servers = ['localhost:9092']
topic_name = 'Topic_test'
#group_id = 'my_group_id1'
consumer = KafkaConsumer(topic_name, bootstrap_servers=bootstrap_servers,
                         auto_offset_reset='earliest', value_deserializer=lambda x: json.loads(x.decode('utf-8')))

In [20]:
import joblib
# Load the saved model
loaded_model = joblib.load('isolation_forest_model.joblib')

In [21]:
import warnings

# Suppress sklearn warning about missing feature names
warnings.filterwarnings("ignore", message="X does not have valid feature names, but IsolationForest")

import numpy as np  # Import NumPy for array manipulation
# Reading messages from the consumer with a 5-second interval
try:
    for message in consumer:
        # Reshape the input data into a 2D array
        sbp = np.array(message.value.get('SBP')).reshape(-1, 1)
        dbp = np.array(message.value.get('DBP')).reshape(-1, 1)

        # Use the loaded model for prediction
        score = loaded_model.decision_function(np.concatenate((sbp, dbp), axis=1))
        anomaly_value = loaded_model.predict(np.concatenate((sbp, dbp), axis=1))
        
        print(message.value)
        print(anomaly_value)

        # 5-second sleep interval
        time.sleep(5)

except KeyboardInterrupt:
    sys.exit()
finally:
    # Close the consumer after reading all messages
    consumer.close()

{'DateTime': '2020-10-24 21:37', 'RR': '12.1', 'SPO2': '94.3', 'MAP': '64.7', 'SBP': '93', 'DBP': '51.2', 'HR': '90.6', 'PP': '41.8', 'CO': '3787.08'}
[1]
{'DateTime': '2020-10-24 21:38', 'RR': '11.8', 'SPO2': '94.1', 'MAP': '61.7', 'SBP': '88.2', 'DBP': '49.2', 'HR': '88.1', 'PP': '39', 'CO': '3435.9'}
[1]
{'DateTime': '2020-10-24 21:39', 'RR': '12.2', 'SPO2': '94', 'MAP': '62.9', 'SBP': '89.6', 'DBP': '49.8', 'HR': '89.2', 'PP': '39.8', 'CO': '3550.16'}
[1]
{'DateTime': '2020-10-24 21:40', 'RR': '13.2', 'SPO2': '93.7', 'MAP': '63.7', 'SBP': '90.8', 'DBP': '50.6', 'HR': '89.6', 'PP': '40.2', 'CO': '3601.92'}
[1]
{'DateTime': '2020-10-24 21:41', 'RR': '12.2', 'SPO2': '93.7', 'MAP': '64', 'SBP': '91.8', 'DBP': '50.8', 'HR': '88.7', 'PP': '41', 'CO': '3636.7'}
[1]
{'DateTime': '2020-10-24 21:42', 'RR': '13.3', 'SPO2': '93.9', 'MAP': '64.5', 'SBP': '91.3', 'DBP': '51.3', 'HR': '86.3', 'PP': '40', 'CO': '3452'}
[1]
{'DateTime': '2020-10-24 21:43', 'RR': '13', 'SPO2': '94.5', 'MAP': '67', '

AttributeError: 'tuple' object has no attribute 'tb_frame'